In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size =  64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = "/content/ara_.txt"

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 74
Number of unique output tokens: 106
Max sequence length for inputs: 47
Max sequence length for outputs: 67


In [ ]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
output_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [ ]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 "'": 3,
 ',': 4,
 '-': 5,
 '.': 6,
 '0': 7,
 '1': 8,
 '2': 9,
 '3': 10,
 '4': 11,
 '5': 12,
 '6': 13,
 '7': 14,
 '8': 15,
 '9': 16,
 ':': 17,
 ';': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'X': 43,
 'Y': 44,
 'Z': 45,
 'a': 46,
 'b': 47,
 'c': 48,
 'd': 49,
 'e': 50,
 'f': 51,
 'g': 52,
 'h': 53,
 'i': 54,
 'j': 55,
 'k': 56,
 'l': 57,
 'm': 58,
 'n': 59,
 'o': 60,
 'p': 61,
 'q': 62,
 'r': 63,
 's': 64,
 't': 65,
 'u': 66,
 'v': 67,
 'w': 68,
 'x': 69,
 'y': 70,
 'z': 71,
 '’': 72,
 '€': 73}

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length,num_decoder_tokens),
    dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, output_token_index[char]] = 1.
    if t > 0:
      decoder_target_data[i, t - 1, output_token_index[char]] = 1.
  decoder_input_data[i, t + 1:, output_token_index[' ']] = 1.
  decoder_target_data[i, t:, output_token_index[' ']] = 1.

In [ ]:
encoder_input_data[0].shape

(47, 74)

In [ ]:
encoder_inputs = Input(shape= (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state= True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape= (None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences= True, return_state= True)
decoder_outputs, _ , _ = decoder_lstm(
    decoder_inputs, initial_state = encoder_states)
deccoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = deccoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer= 'rmsprop', loss= 'categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split = 0.2)

Epoch 1/100
125/125 [==============================] - 8s 27ms/step - loss: 1.2174 - accuracy: 0.7437 - val_loss: 1.6366 - val_accuracy: 0.6147
Epoch 2/100
125/125 [==============================] - 2s 14ms/step - loss: 0.9711 - accuracy: 0.7538 - val_loss: 1.4787 - val_accuracy: 0.6198
Epoch 3/100
125/125 [==============================] - 2s 14ms/step - loss: 0.9066 - accuracy: 0.7667 - val_loss: 1.3937 - val_accuracy: 0.6346
Epoch 4/100
125/125 [==============================] - 2s 15ms/step - loss: 0.8493 - accuracy: 0.7767 - val_loss: 1.3166 - val_accuracy: 0.6518
Epoch 5/100
125/125 [==============================] - 2s 19ms/step - loss: 0.8010 - accuracy: 0.7899 - val_loss: 1.2665 - val_accuracy: 0.6630
Epoch 6/100
125/125 [==============================] - 2s 15ms/step - loss: 0.7723 - accuracy: 0.7948 - val_loss: 1.2542 - val_accuracy: 0.6664
Epoch 7/100
125/125 [==============================] - 2s 14ms/step - loss: 0.7519 - accuracy: 0.7998 - val_loss: 1.2053 - val_accuracy: